In [8]:
# Load Libraries
import pandas as pd
import os
import datetime
import json

In [9]:
from gremlin_python.structure.graph import Graph
from gremlin_python.process.traversal import T
from gremlin_python.process.graph_traversal import __
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.aiohttp.transport import AiohttpTransport

In [10]:
def load_vertex(fdataset_name, fdata_bucket_name):
    print("loading vertex " + fdataset_name)
    
    # Load data from json
    data_location = '{}/{}'.format(fdata_bucket_name, fdataset_name)

    # Load json to datafrase
    vertex_data = {}
    with open(data_location, 'r') as jf:
        vertex_data = json.load(jf)

    # Load to graph
    for data in vertex_data:
        vertex_id = data['id']
        label = data['label']
        properties = data['properties']
        name = data['id']

        #Insert with PK
        vertex = g.addV(label).property(T.id, vertex_id)
        # Insert without PK (duplicate data)
        #vertex = g.addV(label).property('id', vertex_id)

        # Add properties to the vertex
        for key, value in properties.items():
            vertex = vertex.property(key, value)

        # Execute the query
        try:
            vertex.next()
        except Exception as error:
          print("Vertex error => An exception occurred:", error)

In [11]:
def load_edge(fdataset_name, fdata_bucket_name):
    print("loading edge " + fdataset_name)
    
    # Load data from json
    data_location = '{}/{}'.format(fdata_bucket_name, fdataset_name)

    # Load json to datafrase
    edge_data = {}
    with open(data_location, 'r') as jf:
        edge_data = json.load(jf)

    #print(edge_data)
    
    # Load to graph
    for data in edge_data:
        from_id = data['from_id']
        to_id = data['to_id']
        edge_label = data['label']
        edge_properties = data['properties']

        # Create the edge with the given label and properties
        edge = g.V(from_id).as_('a').V(to_id).addE(edge_label).from_('a')

        # Add properties to the edge
        for key, value in edge_properties.items():
            edge = edge.property(key, value)
        
        # Execute the query
        try:
            edge.next()
        except Exception as error:
          print("Edge error => An exception occurred:", error , "|" , edge )

In [12]:
# Prepare connection
endpoint = 'ws://127.0.0.1:8182/gremlin'
connection = DriverRemoteConnection(endpoint,'g',
                 transport_factory=lambda:AiohttpTransport(call_from_event_loop=True))

graph = Graph()
g = graph.traversal().withRemote(connection)

# Load data

In [13]:
data_bucket_name = "/mnt/c/Eliezer/workspace/github.com/jupyter-notebook/graph/dataset"

dataset_name = 'person_vertex.json'
load_vertex(dataset_name, data_bucket_name)

dataset_name = 'account_vertex.json'
load_vertex(dataset_name, data_bucket_name)

dataset_name = 'card_vertex.json'
load_vertex(dataset_name, data_bucket_name)

dataset_name = 'payment_vertex.json'
load_vertex(dataset_name, data_bucket_name)

dataset_name = 'terminal_vertex.json'
load_vertex(dataset_name, data_bucket_name)

dataset_name = 'merchant_vertex.json'
load_vertex(dataset_name, data_bucket_name)

loading vertex person_vertex.json
loading vertex terminal_vertex.json
loading vertex merchant_vertex.json
loading vertex account_vertex.json
loading vertex card_vertex.json
loading vertex payment_vertex.json


# Load Edge

In [14]:
dataset_name = 'person-account-edge.json'
load_edge(dataset_name, data_bucket_name)

dataset_name = 'account-card-edge.json'
load_edge(dataset_name, data_bucket_name)

dataset_name = 'card-payment-edge.json'
load_edge(dataset_name, data_bucket_name)

dataset_name = 'payment-mcc-edge.json'
load_edge(dataset_name, data_bucket_name)

dataset_name = 'payment-terminal-edge.json'
load_edge(dataset_name, data_bucket_name)

loading edge person-account-edge.json
loading edge account-card-edge.json
loading edge card-payment-edge.json
loading edge payment-mcc-edge.json
loading edge payment-terminal-edge.json


# Query

In [70]:
for i in range(len(data_person)):
    
    vertex=data_person.iloc[i, 2]
    id=data_person.iloc[i, 1]
    name=data_person.iloc[i, 3]
    
    print('data : ',id, vertex, name)
    
    person = g.V(vertex).property('name',name).as_(id)
person.next()

data :  P-1 Person-1 20
data :  P-2 Person-2 21
data :  P-3 Person-3 24
data :  P-4 Person-4 20
data :  P-5 Person-5 21
data :  P-6 Person-6 24
data :  P-7 Person-7 24
data :  P-8 Person-8 25
data :  P-9 Person-9 20
data :  P-10 Person-10 20
data :  P-11 Person-11 22
data :  P-12 Person-12 20
data :  P-13 Person-13 19
data :  P-14 Person-14 18
data :  P-15 Person-15 24
data :  P-16 Person-16 18
data :  P-17 Person-17 20
data :  P-18 Person-18 21
data :  P-19 Person-19 18
data :  P-20 Person-20 23
data :  P-21 Person-21 21
data :  P-22 Person-22 25
data :  P-23 Person-23 22
data :  P-24 Person-24 24
data :  P-25 Person-25 18
data :  P-26 Person-26 19
data :  P-27 Person-27 22
data :  P-28 Person-28 21
data :  P-29 Person-29 20
data :  P-30 Person-30 23
data :  P-31 Person-31 19
data :  P-32 Person-32 25
data :  P-33 Person-33 24
data :  P-34 Person-34 19
data :  P-35 Person-35 24
data :  P-36 Person-36 18
data :  P-37 Person-37 19
data :  P-38 Person-38 20
data :  P-39 Person-39 24
data

KeyboardInterrupt: 

In [56]:
connection.close()

KeyboardInterrupt: 

In [64]:
x = g.V('P-2354').valueMap().next()
print(x)

StopIteration: 